##### [ 한글 데이터셋  RNN ] <hr>
- 데이터셋 : Kopora의 NAVER Sentiment Movie Corpus

- [1] 데이터 준비 <hr>

In [ ]:
#### ===> 모듈 로딩
from Korpora import Korpora 
import pandas as pd 
import numpy as np 

In [ ]:
#### ===> 데이터 로딩
corpous = Korpora.load('nsmc')
print(corpous)

In [ ]:
nsmcDF = pd.DataFrame(corpous.test)
nsmcDF.info()

In [ ]:
nsmIter=nsmcDF.itertuples()
for item in nsmIter:
    print(item, item.text, item.label)
    break

- [2] 데이터셋 준비 <hr>

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, corpus):
        nsmcDF = pd.DataFrame(corpus).fillna('')

        x_data = nsmcDF['text'].values
        self.x_data = x_data
        self.y_data = nsmcDF['label'].values

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return y, x

In [ ]:
# class CustomDataset(Dataset):
#     def __init__(self, texts, labels, vocab, tokenizer):
#         super().__init__()
#         self.texts = texts
#         self.labels = labels
#         self.vocab = vocab
#         self.tokenizer = tokenizer
        
        
#     def __len__(self):
#         return len(self.labels)
        
#     def __getitem__(self, idx):
#         text = self.texts.iloc[idx]
#         label = self.labels.iloc[idx]
#         return self.vocab(self.tokenizer(text)), label

In [ ]:
trainDS = CustomDataset(corpous.train)
testDS = CustomDataset(corpous.test)

In [ ]:
print(f'trainDS =>  {len(trainDS)}개   testDS =>  {len(testDS)}개')

In [ ]:
for label, text in trainDS:
    print(label, text)
    break

for label, text in testDS:
    print(label, text)
    break

- [2] 단어 사전 생성 <hr>
    * 토큰화 진행 ==> 형태소 분석기 선택 
    * 단어 사전 

- [2-1] 토큰화 관련 준비

In [ ]:
### ===> 모듈 로딩
from konlpy.tag import Okt
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
### ===> 토큰관련 특별 문자
UNK = '<UNK>'
PAD = '<PAD>'

In [ ]:
### 토큰화 인스턴스 생성
tokenizer = Okt()

In [ ]:
### ===> 토큰 제너레이터 함수 : 데이터 추출하여 토큰화 
def yield_tokens(data_iter):
    for label, text in data_iter:
        # 라벨, 텍스트 --> 텍스트 토큰화
        yield tokenizer.morphs(text, stem=True)

- [2-2] 토큰화 ===> 단어/어휘 사전 생성

In [ ]:
### ===> 토큰화 및 단어/어휘 사전 생성
VOCAB = build_vocab_from_iterator(
    yield_tokens(trainDS),
    min_freq=2,
    specials= [PAD, UNK],
    special_first=True
)

### <UNK> 인덱스 설정
VOCAB.set_default_index(VOCAB[UNK])

In [ ]:
### VOCAB 메서드 
VOCAB.get_itos()[:9], VOCAB.get_stoi()['영화'], VOCAB.get_stoi()[UNK]

In [ ]:
### ===> 텍스트 >>>> 정수 인코딩
text_pipeline = lambda x: VOCAB(tokenizer.morphs(text, stem=True))

### ===> 레이틀 >>> 정수 인코딩 (0~3)
label_pipeline = lambda x: int(x) - 1

- [2-3] 인코딩 & 디코딩 인덱싱 

In [ ]:
### 인코딩 : 문자 >>>> 숫자로 변환
token_to_id ={ label : id  for label, id in VOCAB.get_stoi().items()}

### 디코딩 : 숫자 >>>> 문자로 변환
id_to_token ={ id : label  for label, id in VOCAB.get_stoi().items()}

- [3] 데이터 로더 생성 <hr>

In [ ]:
### ===> 모듈로딩
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pad_sequence
import torch  


In [ ]:
### ===> 실행 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
### ===> DataLoader에서 배치크기만큼 데이터셋 반환 함수 
def collate_batch(batch):
    # 배치크기 만큼의 라벨, 텍스트, 오프셋 값 저장 변수 
    label_list, text_list= [], []
    
    # 1개씩 뉴스기사, 라벨 추출 해서 저장 
    for (_label, _text) in batch:
         # 라벨 인코딩 후 저장
         label_list.append(label_pipeline(_label))
         
         # 텍스트 인코딩 후 저장
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         
    
    # 텐서화 진행     
    label_list = torch.tensor(label_list, dtype=torch.float32)
    
    # 문장의 길이 일치
    text_list = pad_sequence(text_list, batch_first=True, padding_value=0)
    
    return label_list.to(device), text_list.to(device)



In [ ]:
### ===> 학습용, 검증용, 테스트용 DataSet 준비 
BATCH_SIZE = 64

### 학습용, 검증용, 테스트용 Dataset, DataLoader 준비
num_train = int(len(trainDS) * 0.95)
print(f' num_train :{num_train}')

split_trainDS, split_validDS= random_split( trainDS, [num_train, len(trainDS) - num_train])
print(f' len(split_trainDS) :{len(split_trainDS)}')
print(f' len(split_validDS) :{len(split_validDS)}')

trainDL = DataLoader( split_trainDS, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch )
validDL = DataLoader( split_validDS, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch )
testDL  = DataLoader( testDS,        batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch )



In [ ]:
print(f' len(trainDL) :{len(trainDL)*BATCH_SIZE}')
print(f' len(validDL) :{len(validDL)*BATCH_SIZE}')
print(f' len(testDL) :{len(testDL)* BATCH_SIZE}')